
# **End-to-End Pipeline**



In this notebook:

- All individual phases of the pipeline are integrated into a unified structure that functions as a complete system  
- The main components include:  
  - **YOLO** for object detection  
  - **KPD** for keypoint detection  
  - **PnP** for 6D pose estimation  
  - **ADD** for pose evaluation  
- Each component is modularized and exported to a separate file for better organization  
- Required modules are imported as needed to ensure clarity, reusability, and maintainable code structure  

Additionally, all trained models are tested on the **test set** to evaluate their accuracy and compare their performance.

## **Importing required libraries and modules**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.7 MB/s eta 0:00:00


In [ ]:
import sys
import os

BASE_DIR = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation"
MODELS = os.path.join(BASE_DIR, "notebooks", "end_to_end", "modules", "models")
UTILS = os.path.join(BASE_DIR, "notebooks", "end_to_end", "modules", "utils")

sys.path.append(MODELS)
sys.path.append(UTILS)

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from abc import ABC, abstractmethod
import pandas as pd


#models
from ultralytics import YOLO
from baseline_model_class import KeypointHeatmapNet
from extended_model_class import CrossFuNet


#yolo functions
from yolo_utils import crop_and_resize, plot_detection_and_crop

#kpd functions
from kpd_utils import extract_keypoints_to_original_image_space

#pnp functions
from pnp_utils import run_pnp

#add functions
from add_utils import evaluate_pose_estimation

### **Data paths**

In [ ]:
#--------MODELS---------

YOLO_PATH = os.path.join(BASE_DIR, "models", "yolov10m", "weights", "best.pt")

#------------BASELINE------------

KPD_FPS = os.path.join(BASE_DIR,"models", "resnet", "baseline_fps_model.pth")
KPD_CPS = os.path.join(BASE_DIR, "models", "resnet", "baseline_cps_model.pth")


#------------EXTENDED------------

KPD_CROSS_FUSION_RELU = os.path.join(BASE_DIR, "models", "resnet", "extended_relu_model.pth")
KPD_CROSS_FUSION_SILU = os.path.join(BASE_DIR, "models","resnet", "extended_silu_model.pth")
KPD_CROSS_FUSION_MISH = os.path.join(BASE_DIR, "models","resnet", "extended_mish_model.pth")

KPD_CROSS_FUSION_RELU_COS = os.path.join(BASE_DIR, "models", "resnet", "extended_relu_cos_model.pth")
KPD_CROSS_FUSION_SILU_COS = os.path.join(BASE_DIR, "models","resnet", "extended_silu_cos_model.pth")
KPD_CROSS_FUSION_MISH_COS = os.path.join(BASE_DIR, "models","resnet", "extended_mish_cos_model.pth")

KPD_CROSS_FUSION_RELU_POLY = os.path.join(BASE_DIR, "models", "resnet", "extended_relu_poly_model.pth")
KPD_CROSS_FUSION_SILU_POLY = os.path.join(BASE_DIR, "models","resnet", "extended_silu_poly_model.pth")
KPD_CROSS_FUSION_MISH_POLY = os.path.join(BASE_DIR, "models","resnet", "extended_mish_poly_model.pth")

#3D Points for performing PnP
KP3D_CPS_JSON = os.path.join(BASE_DIR, "data/point_sampling_data/3D_50_keypoints_cps.json")
KP3D_FPS_JSON = os.path.join(BASE_DIR, "data/point_sampling_data/3D_50_keypoints_fps.json")

#GT TEST DATA
GT_JSON = os.path.join(BASE_DIR, "data/full_data/test/gt.json")

#TEST_IMAGES
TEST_IMAGES = os.path.join(BASE_DIR, "data/full_data/test/images")

#DEPTH TEST IMAGES
TEST_DEPTH_IMAGES = os.path.join(BASE_DIR, "data/full_data/test/depth")

### **Models definition**

- RGBPoseEstimator - baseline model that uses only RGB images

In [ ]:
class RGBPoseEstimator:

    def __init__(self, yolo_model_path, kpd_model_path, kp3d_path, num_keypoints=50):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        #loading models on gpu -> for yolo it is by default
        self.yolo = YOLO(yolo_model_path)
        self.kpd = self.load_kpd(kpd_model_path, num_keypoints)
        self.kpd.to(self.device).eval()

        #loading 3D points needed for PnP
        self.kp3d_dict = self.load_3D_points(kp3d_path)

        #normalization needed for ResNet Network
        self.to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std =[0.229, 0.224, 0.225]
        )

    #loading kpd
    def load_kpd(self, path, num_keypoints):
        model = KeypointHeatmapNet(num_keypoints=num_keypoints)
        state_dict = torch.load(path, map_location=self.device)
        model.load_state_dict(state_dict)
        return model

    #loading 3D points
    def load_3D_points(self, kp3d_path):
        with open(kp3d_path, "r") as f:
            kp3d_50 = json.load(f)
        return kp3d_50

    #YOLO functions
    def detect_object(self, image_path):
        return self.yolo(image_path)

    def get_bbox(self, result, index=0):
        box = result[0].boxes[index].xyxy[0].tolist()
        return list(map(int, box))

    #KPD functions
    def preprocess_image_for_kpd(self, image_path, bbox, size=(256, 256)):

        image = Image.open(image_path).convert("RGB")
        image_tensor = self.to_tensor(image)
        x1, y1, x2, y2 = bbox #msm da nema potrebe dradit ponovo map

        # Crop and resize
        crop = image_tensor[:, y1:y2, x1:x2]
        resized = torch.nn.functional.interpolate(
            crop.unsqueeze(0), size=size, mode="bilinear", align_corners=False
        )

        # Normalize for ResNet
        normalized = self.normalize(resized.squeeze(0)).unsqueeze(0).to(self.device)
        return normalized

    #KPD functions
    def detect_keypoints(self, image_path, bbox):
        image_tensor = self.preprocess_image_for_kpd(image_path, bbox)
        heatmaps = self.kpd(image_tensor)
        return heatmaps

    #Estimating pose for one picture
    def estimate_pose(self, image_path):

        #yolo outputs
        result = self.detect_object(image_path)
        bbox = self.get_bbox(result)

        #kpd outputs
        heatmaps = self.detect_keypoints(image_path, bbox)
        keypoints_2d = extract_keypoints_to_original_image_space(heatmaps, bbox)

        #taking only xx_xxxx.png
        img_id = image_path.split("/")[-1]
        obj_key, pose_result, inliers = run_pnp(img_id, keypoints_2d.tolist(), self.kp3d_dict)


        return img_id.split(".")[0], pose_result, inliers


    #If we want to perform evaluation, process is paralelized
    def __paralelized_estimate_pose(self, image_paths):

      def process_image(img_path):
          try:
              img_id, pose_result, inliers = self.estimate_pose(img_path)
              if pose_result is not None:
                  return img_id, pose_result, inliers
          except Exception as e:
              print(f"Error processing {img_path}: {e}")
          return None

      pnp_results = {}
      skipped = []

      # Use ThreadPoolExecutor for parallel inference
      with ThreadPoolExecutor(max_workers=4) as executor:
          futures = {executor.submit(process_image, img_path): img_path for img_path in image_paths}
          for future in as_completed(futures):

              result = future.result()
              if result:
                  img_id, pose_result, inliers = result
                  pnp_results[img_id] = (pose_result, inliers)
              else:
                  img_path = futures[future]
                  skipped.append(img_path)

      return pnp_results, skipped


    #method for evaluation of test data
    def evaluate(self, image_paths, GT_JSON, diameter_map, symmetric_objects, threshold_ratio=0.1, debug=False):

        with open(GT_JSON, "r") as f:
            gt_data = json.load(f)

        pnp_results, skipped = self.__paralelized_estimate_pose(image_paths)

        if debug:
            print(f"[INFO] Processed {len(pnp_results)} images.")
            print(f"[INFO] Example keys: {list(pnp_results.keys())[:5]}")
            print(f"[INFO] Number of skipped examples: {len(skipped)}")

        accuracy_results, results_distribution_class, high_error_samples = evaluate_pose_estimation(
            pnp_results=pnp_results,
            kp3d=self.kp3d_dict,
            gt_data=gt_data,
            diameter_map=diameter_map,
            symmetric_objects=symmetric_objects,
            threshold_ratio=threshold_ratio
        )

        return accuracy_results, results_distribution_class, high_error_samples

- RGBDepthPoseEstimator - extended model that used RBG and depth images

In [ ]:
class RGBDPoseEstimator:

    def __init__(self, yolo_model_path, kpd_model_path, kpd_class, act_layer, kp3d_path, num_keypoints=50):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        #loading models on gpu -> for yolo it is by default
        self.yolo = YOLO(yolo_model_path)
        self.kpd = self.load_kpd(kpd_model_path, kpd_class, act_layer, num_keypoints)
        self.kpd.to(self.device).eval()

        #loading 3D points needed for PnP
        self.kp3d_dict = self.load_3D_points(kp3d_path)

        #normalization needed for ResNet Network
        self.to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std =[0.229, 0.224, 0.225]
        )

    #loading kpd
    def load_kpd(self, path, kpd_class, act_layer, num_keypoints):
        model = kpd_class(act_layer = act_layer, num_keypoints = num_keypoints) #Model with DEPTH
        state_dict = torch.load(path, map_location=self.device)
        model.load_state_dict(state_dict, strict = False)
        return model

    #loading 3D points
    def load_3D_points(self, kp3d_path): #SAME
        with open(kp3d_path, "r") as f:
            kp3d_50 = json.load(f)
        return kp3d_50

    #YOLO functions
    def detect_object(self, image_path): #SAME
        return self.yolo(image_path)

    def get_bbox(self, result, index=0): #SAME
        box = result[0].boxes[index].xyxy[0].tolist()
        return list(map(int, box))


    #KPD functions
    def preprocess_image_for_kpd(self, image_path, bbox, size=(256, 256)):

        image = Image.open(image_path).convert("RGB")
        image_tensor = self.to_tensor(image)
        x1, y1, x2, y2 = bbox #msm da nema potrebe dradit ponovo map

        # Crop and resize
        crop = image_tensor[:, y1:y2, x1:x2]
        resized = torch.nn.functional.interpolate(
            crop.unsqueeze(0), size=size, mode="bilinear", align_corners=False
        )

        # Normalize for ResNet
        normalized = self.normalize(resized.squeeze(0)).unsqueeze(0).to(self.device)

        return normalized


    def preprocess_depth_image_for_kpd(self, depth_path, bbox, size=(256, 256)):

        #open depth image
        depth_img = Image.open(depth_path)
        depth_np = np.array(depth_img).astype(np.float32)  # [H, W]

        #normalize it for resnet
        depth_np = (depth_np - 500.0) / 1000.0
        depth_np = np.clip(depth_np, 0.0, 1.0)

        depth_tensor = torch.from_numpy(depth_np).unsqueeze(0)

        # crop by bounding box
        x1, y1, x2, y2 = bbox
        depth_crop = depth_tensor[:, y1:y2, x1:x2]

        # rescale to 256x256
        resized = torch.nn.functional.interpolate(
            depth_crop.unsqueeze(0), size=size, mode="bilinear", align_corners=False
        )  # [1, 1, H, W]


        # only 1 channe not like RGB!!!
        return resized.to(self.device)

    #KPD functions
    def detect_keypoints(self, image_path, depth_path, bbox):  #A bit different

        image_tensor = self.preprocess_image_for_kpd(image_path, bbox)
        depth_tensor = self.preprocess_depth_image_for_kpd(depth_path, bbox)
        heatmaps = self.kpd(image_tensor, depth_tensor)
        return heatmaps


    #Estimating pose for one picture
    def estimate_pose(self, image_path, depth_path):

        #yolo outputs
        result = self.detect_object(image_path)
        bbox = self.get_bbox(result)

        #kpd outputs
        heatmaps = self.detect_keypoints(image_path, depth_path, bbox)
        keypoints_2d = extract_keypoints_to_original_image_space(heatmaps, bbox)

        #taking only xx_xxxx.png
        img_id = image_path.split("/")[-1]
        obj_key, pose_result, inliers = run_pnp(img_id, keypoints_2d.tolist(), self.kp3d_dict)


        return img_id.split(".")[0], pose_result, inliers


    #If we want to perform evaluation, process is paralelized
    def __paralelized_estimate_pose(self, image_paths, depth_paths):

      def process_image(img_path, depth_path):
          try:
              img_id, pose_result, inliers = self.estimate_pose(img_path, depth_path)
              if pose_result is not None:
                  return img_id, pose_result, inliers
          except Exception as e:
              print(f"Error processing {img_path}: {e}")
          return None

      pnp_results = {}
      skipped = []

      # Use ThreadPoolExecutor for parallel inference
      with ThreadPoolExecutor(max_workers=4) as executor:
          futures = {}
          # we send sorted paths
          for i, img_path in enumerate(image_paths):

              depth_path = depth_paths[i]
              futures[executor.submit(process_image, img_path, depth_path)] = img_path

          for future in as_completed(futures):
              result = future.result()
              if result:
                  img_id, pose_result, inliers = result
                  pnp_results[img_id] = (pose_result, inliers)
              else:
                  img_path = futures[future]
                  skipped.append(img_path)

      return pnp_results, skipped


    #method for evaluation of test data
    def evaluate(self, image_paths, depth_paths, GT_JSON, diameter_map, symmetric_objects, threshold_ratio=0.1, debug=False):

        with open(GT_JSON, "r") as f:
            gt_data = json.load(f)

        pnp_results, skipped = self.__paralelized_estimate_pose(image_paths, depth_paths)

        if debug:
            print(f"[INFO] Processed {len(pnp_results)} images.")
            print(f"[INFO] Example keys: {list(pnp_results.keys())[:5]}")
            print(f"[INFO] Number of skipped examples: {len(skipped)}")

        accuracy_results, results_distribution_class, high_error_samples = evaluate_pose_estimation(
            pnp_results=pnp_results,
            kp3d=self.kp3d_dict,
            gt_data=gt_data,
            diameter_map=diameter_map,
            symmetric_objects=symmetric_objects,
            threshold_ratio=threshold_ratio
        )

        return accuracy_results, results_distribution_class, high_error_samples

# **Evaluation part**

In [ ]:
BASELINE_ESTIMATORS = {

    # ------------ BASELINE ------------
    "baseline_fps": RGBPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_FPS,
        kp3d_path=KP3D_FPS_JSON
    ),

    "baseline_cps": RGBPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CPS,
        kp3d_path=KP3D_CPS_JSON
    ),


}



EXTENDED_ESTIMATORS = {

    # ------------ EXTENDED ------------
    "cross_fusion_relu": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_RELU,
        kpd_class=CrossFuNet,
        act_layer=nn.ReLU(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    "cross_fusion_silu": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_SILU,
        kpd_class=CrossFuNet,
        act_layer=nn.SiLU(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    "cross_fusion_mish": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_MISH,
        kpd_class=CrossFuNet,
        act_layer=nn.Mish(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    # ------------ EXTENDED CROSS FUSION COSINE ------------
    "cross_fusion_relu_cos": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_RELU_COS,
        kpd_class=CrossFuNet,
        act_layer=nn.ReLU(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    "cross_fusion_silu_cos": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_SILU_COS,
        kpd_class=CrossFuNet,
        act_layer=nn.SiLU(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    "cross_fusion_mish_cos": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_MISH_COS,
        kpd_class=CrossFuNet,
        act_layer=nn.Mish(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    # ------------ EXTENDED CROSS FUSION POLYNOMIAL ------------
    "cross_fusion_relu_poly": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_RELU_POLY,
        kpd_class=CrossFuNet,
        act_layer=nn.ReLU(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    "cross_fusion_silu_poly": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_SILU_POLY,
        kpd_class=CrossFuNet,
        act_layer=nn.SiLU(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    ),

    "cross_fusion_mish_poly": RGBDPoseEstimator(
        yolo_model_path=YOLO_PATH,
        kpd_model_path=KPD_CROSS_FUSION_MISH_POLY,
        kpd_class=CrossFuNet,
        act_layer=nn.Mish(inplace=True),
        kp3d_path=KP3D_FPS_JSON
    )
}


## **Execution**

In [ ]:
test_images = sorted(os.listdir(TEST_IMAGES))
depth_images  = sorted(os.listdir(TEST_DEPTH_IMAGES))
diameter_map = {
    '01': 102.09865663,
    '02': 247.50624233,
    '03': 167.35486092,
    '04': 172.49224865,
    '05': 201.40358597,
    '06': 154.54551808,
    '07': 124.26430816,
    '08': 261.47178102,
    '09': 108.99920102,
    '10': 164.62758848,
    '11': 175.88933422,
    '12': 145.54287471,
    '13': 278.07811733,
    '14': 282.60129399,
    '15': 212.35825148
}

symmetric_objects = {'10', '11'}  #eggbox (10) and glue (11) are symmetric objects in linemod dataset
image_paths = [f"{TEST_IMAGES}/{image}"for image in test_images]
depth_paths = [f"{TEST_DEPTH_IMAGES}/{depth_image}"for depth_image in depth_images]

In [ ]:
RESULTS = {}

for key, estimator in EXTENDED_ESTIMATORS.items():
  accuracy, _, _ = estimator.evaluate(image_paths, depth_paths, GT_JSON, diameter_map, symmetric_objects)
  RESULTS[key] = accuracy

for key, estimator in BASELINE_ESTIMATORS.items():
  accuracy, _, _ = estimator.evaluate(image_paths, GT_JSON, diameter_map, symmetric_objects)
  RESULTS[key] = accuracy


Streaming output truncated to the last 5000 lines.

image 1/1 /content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/full_data/test/images/15_0436.png: 480x640 1 phone, 28.4ms
Speed: 8.3ms preprocess, 28.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)


image 1/1 /content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/full_data/test/images/15_0444.png: 480x640 1 phone, 26.7ms
Speed: 2.1ms preprocess, 26.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
image 1/1 /content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/full_data/test/images/15_0448.png: 480x640 1 phone, 20.0ms
Speed: 1.9ms preprocess, 20.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)


image 1/1 /content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/full_data/test/images/15_0458.png: 480x640 1 phone, 28.6ms
Speed: 2.1ms preprocess, 28.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
image 1/1 /content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/full_data/

In [ ]:
df = pd.DataFrame(RESULTS)
df.loc['Mean'] = df.mean()
df

,baseline_fps,baseline_cps,cross_fusion_relu,cross_fusion_silu,cross_fusion_mish,cross_fusion_relu_cos,cross_fusion_silu_cos,cross_fusion_mish_cos,cross_fusion_relu_poly,cross_fusion_silu_poly,cross_fusion_mish_poly
01,52.419355,47.580645,62.096774,60.483871,68.548387,73.387097,75.806452,68.548387,62.903226,59.677419,65.322581
02,92.561983,90.082645,95.041322,96.694215,92.561983,97.520661,98.347107,96.694215,95.041322,93.388430,92.561983
04,90.833333,82.500000,95.000000,89.166667,85.833333,92.500000,94.166667,96.666667,89.166667,89.166667,90.000000
05,90.000000,91.666667,91.666667,90.000000,90.833333,93.333333,96.666667,92.500000,92.500000,86.666667,89.166667
06,77.118644,82.203390,89.830508,76.271186,83.898305,93.220339,83.050847,88.983051,87.288136,81.355932,82.203390
08,96.638655,96.638655,95.798319,93.277311,96.638655,98.319328,94.957983,97.478992,93.277311,92.436975,95.798319
09,61.600000,60.000000,78.400000,68.800000,77.600000,80.000000,77.600000,83.200000,72.800000,67.200000,72.000000
10,87.200000,89.600000,90.400000,86.400000,91.200000,96.000000,94.400000,91.200000,87.200000,88.000000,88.800000
11,86.885246,88.524590,95.901639,91.803279,95.081967,95.901639,94.262295,96.721311,94.262295,90.163934,89.344262
12,83.870968,79.032258,83.870968,85.483871,87.903226,93.548387,94.354839,93.548387,89.516129,82.258065,82.258065
